## 1. Парсим новости с сайта Lenta.ru

In [ ]:
# Установка библиотек
%pip install bs4 openpyxl webdriver-manager tqdm mutliprocess pandas numpy lxml

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import requests as rq
from IPython import display
from tqdm import tqdm
from multiprocess import Pool

In [ ]:
  # url = 'https://lenta.ru/search/v2/process?'
  #       + 'from=0&'\                       # Смещение
  #       + 'size=1000&'\                    # Кол-во статей
  #       + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
  #       + 'title_only=0&'\                 # Точная фраза в заголовке
  #       + 'domain=1&'\                     # ??
  #       + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
  #       + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
  #       + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
  #       + 'modified%2Cfrom=2020-01-01&'\
  #       + 'modified%2Cto=2020-11-01&'\
  #       + 'query='                         # Поисковой запрос

In [ ]:
class lentaRu_parser:
    def __init__(self):
        pass

    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = (
            'https://lenta.ru/search/v2/process?'
            + 'from={}&'.format(param_dict['from'])
            + 'size={}&'.format(param_dict['size'])
            + 'sort={}&'.format(param_dict['sort'])
            + 'title_only={}&'.format(param_dict['title_only'])
            + 'domain={}&'.format(param_dict['domain'])
            + 'modified%2Cformat=yyyy-MM-dd&'
        )

        # Добавляем условные параметры только если они нужны
        if hasType:
            url += 'type={}&'.format(param_dict['type'])
        if hasBloc:
            url += 'bloc={}&'.format(param_dict['bloc'])

        url += (
            'modified%2Cfrom={}&'.format(param_dict['dateFrom'])
            + 'modified%2Cto={}&'.format(param_dict['dateTo'])
            # + 'query={}'.format(param_dict['query'])
        )

        return url

    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        r.raise_for_status()  # полезно для явной обработки ошибок
        search_table = pd.DataFrame(r.json()['matches'])
        return search_table

    def get_articles(
        self,
        param_dict,
        time_step=37,
        save_every=5,
        save_excel=True
    ) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней
        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')

            print(
                'Parsing articles from '
                + param_copy['dateFrom'] + ' to ' + param_copy['dateTo']
            )

            chunk_df = self._get_search_table(param_copy)

            out = pd.concat([out, chunk_df], ignore_index=True)

            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1

            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx", index=False)
                print('Checkpoint saved!')
                save_counter = 0

        if save_excel:
            out.to_excel(
                "lenta_{}_{}.xlsx".format(
                    param_dict['dateFrom'], param_dict['dateTo']
                ),
                index=False
            )
        print('Finish')
        return out


In [ ]:
lenta_topics = {
    'Общество/Россия': '1',
    'Экономика': '4',
    'Силовые структуры': '37',
    'Бывший СССР': '3',
    'Спорт': '8',
    'Забота о себе': '87',
    # 'Строительство': '',
    'Туризм/Путешествия': '48',
    'Наука и техника': '5',
    }


In [ ]:
# Задаем тут параметры
query = ''
offset = 0
size = 100
sort = "1"
title_only = "0"
domain = "1"
material = "0"
bloc = "0" # topic = тематика новости
dateFrom = '2023-01-01'
dateTo = "2025-12-30"

param_dict = {
    'query'     : query,
    'from'      : str(offset),
    'size'      : str(size),
    'dateFrom'  : dateFrom,
    'dateTo'    : dateTo,
    'sort'      : sort,
    'title_only': title_only,
    'type'      : material,
    'bloc'      : bloc,
    'domain'    : domain
  }

print("param_dict:", param_dict)

param_dict: {'query': '', 'from': '0', 'size': '100', 'dateFrom': '2023-01-01', 'dateTo': '2025-12-30', 'sort': '1', 'title_only': '0', 'type': '0', 'bloc': '0', 'domain': '1'}


In [ ]:

parser = lentaRu_parser()

# по каждой теме соберем отдельный датасет, а потом объемдиним их в один
tbls = []
for topic in tqdm(lenta_topics):
    param_dict['bloc'] = lenta_topics[topic]
    tbl = parser.get_articles(param_dict=param_dict,
                              time_step = 30,
                              save_every = 5,
                              save_excel = True)
    tbl['topic'] = topic
    tbls.append(tbl)

Checkpoint saved!
Parsing articles from 2025-12-21 to 2025-12-30


100%|██████████| 8/8 [08:42<00:00, 65.31s/it]

Finish


In [ ]:
list(map(lambda x: x.shape, tbls))

[(3600, 17),
 (3600, 17),
 (3600, 17),
 (3600, 17),
 (3582, 17),
 (3567, 17),
 (3600, 17),
 (3587, 17)]

In [ ]:
tbl = pd.concat(tbls, ignore_index=True)
tbl.shape

(28736, 17)

In [ ]:
pd.concat([tbl.value_counts('bloc'),
           tbl.value_counts('bloc', normalize=True)], axis=1)

,count,proportion
bloc,,
1,3600,0.125278
3,3600,0.125278
4,3600,0.125278
37,3600,0.125278
48,3600,0.125278
5,3587,0.124826
8,3582,0.124652
87,3567,0.124130


Распределение текстов по темам получилось равномерное, но тут не хватает темы "строительтсво", поэтому достанем статьи по этой теме из РБК. Также по предварительному анализу я увидел, что темы "Россия" и "Экономика" классификатор распознает хуже всего, поэтому их загружен побольше.

In [ ]:
tbl.to_excel('lenta_arts.xlsx')

## 1. Парсим новости с сайта РБК.ру

In [ ]:
import time
import re
import datetime
import asyncio
from dataclasses import dataclass
from multiprocessing import Pool

from multiprocess import Pool as mPool
import aiohttp
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import nest_asyncio
nest_asyncio.apply()

DEPTH = 350

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
# chrome_options.add_argument("headless")  # в режиме закрытого окна селениум не может считать РБК
chrome_options.add_argument("no-sandbox")
chrome_options.add_argument("disable-dev-shm-usage")

In [ ]:
@dataclass
class Article:
    topic: str = None
    url: str = None
    title: str = None
    content: str = None
    datetime: str = None

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
# сбор ссылок статей с основной страницы для дальнейшего парсинга
def get_pages():

    last_count = 0
    scroll_attempts = 0
    max_no_new_data_attempts = 3

    for _ in tqdm(range(DEPTH), leave=False):
        html = driver.page_source
        soup = BeautifulSoup(html, features="lxml")
        articles = soup.find_all('a', {'class': 'search-item__link js-search-item-link'})
        current_count = len(articles)

        if current_count > last_count:
            # Количество статей увеличилось
            print(f"Статей: {last_count} → {current_count} (+{current_count - last_count})")
            last_count = current_count
            scroll_attempts = 0
        else:
            # Количество статей не изменилось
            scroll_attempts += 1
            print(f"Количество статей не изменилось: {current_count}. Попытка {scroll_attempts}/{max_no_new_data_attempts}")

            if scroll_attempts >= max_no_new_data_attempts:
                print("Загрузка новых данных остановилась, завершаем скроллинг")
                break

        driver.execute_script(
            f"window.scrollTo(0, document.body.scrollHeight - 1200)"
        )
        time.sleep(0.5)

    html = driver.page_source
    soup = BeautifulSoup(html, features="lxml")
    articles = soup.find_all('a', {'class': 'search-item__link js-search-item-link'})
    a_links = [a['href'] for a in articles]
    return a_links

Часть топиков как таковых на РБК отсутствует, поэтому для таких случае применяется поиск по тэгу. Только категория "забота о себе" не попадает ни в ту, ни в другую группу, поэтому ее с РБК было решено не выгружать.

In [ ]:
rbc_topics = {
    'Общество/Россия': 'society',
    'Экономика': 'economics',
    'Бывший СССР': '',
    'Спорт': 'sport',
    'Строительство': '',
    'Туризм/Путешествия': '',
}

rbc_url = 'https://www.rbc.ru/search/?project=rbcnews&dateFrom=01.01.2023&dateTo=31.12.2025&category={}'
# тут менял год dateTo, чтобы выгрузить дополнительно статей, т.к есть ограничение в 2000 шт


not_found_topics =     {'Туризм/Путешествия': ['https://www.rbc.ru/tags/?&dateFrom=01.01.2023&dateTo=31.12.2025&tag=туризм',
                                               'https://www.rbc.ru/tags/?&dateFrom=01.01.2023&dateTo=31.12.2025&tag=путешествия'],
                        'Бывший СССР':         'https://www.rbc.ru/tags/?&dateFrom=01.01.2023&dateTo=31.12.2025&tag=СССР',
                        'Строительство':       'https://www.rbc.ru/search/?query=&project=realty&dateFrom=01.01.2023&dateTo=31.12.2025',
                        }

topic_urls = {}
for topic in rbc_topics:
    if rbc_topics[topic]:
        topic_urls[topic] = rbc_url.format(rbc_topics[topic])
    else:
        topic_urls[topic] = not_found_topics[topic]



In [ ]:
# needed_topics = topic_urls.keys()
# я дополнительно второй раз скачал статьи ранее 2025 года, т.к. у РБК стоит ограничение при скроллинге до 2000 статей.
needed_topics = ['Общество/Россия', 'Экономика']

needed_topic_urls = {topic: topic_urls[topic] for topic in needed_topics}

In [ ]:
# неподсредственный сбор ссылок
art_dct = {}
for topic, urls in tqdm(needed_topic_urls.items(), leave=False):
  art_tmp = []
  print(topic)

  if isinstance(urls, list):
    for url in tqdm(urls):
      driver.get(url)
      arts = get_pages()
      art_tmp.extend(arts)

  else:
    driver.get(urls)
    arts = get_pages()
    art_tmp.extend(arts)

  art_dct[topic] = art_tmp

In [ ]:
art_dct = {key: list(set(art_dct[key])) for key in art_dct}
pd.Series({key: len(set(value)) for key, value in art_dct.items()}, name='count')

In [ ]:
# для парсинга отдельный статей selenium работает медленно, поэтому решил добавить немного асинхронности

async def fetch(session, url):
    try:
        async with session.get(url, timeout=3) as response:
            return url, await response.text()
    except TimeoutError:
        return '', ''


async def fetch_all(urls):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Google Chrome";v="143", "Chromium";v="143", "Not A(Brand";v="24"',
        }
    async with aiohttp.ClientSession(headers=headers) as session:
        results = await asyncio.gather(*[fetch(session, url) for url in urls])
        return results


def parse_article(url, html):

    soup = BeautifulSoup(html, features='lxml')
    try:
        art_content = soup.find('div', {'class': 'l-col-center-590 article__content'})
        title = art_content.find('h1', {'class': lambda x: re.match(r'article__header__title.*', x)}).text.strip()

        main_content = art_content.find('div', {'class': 'article__text article__text_free'})
        text = main_content.find_all('p')
        text = ' '.join([p.text.strip() for p in text if p.text.strip()])

        date = art_content.find('time', {'class': 'article__header__date'})
        date = datetime.datetime.fromisoformat(date['datetime'])

        article = Article(topic=None, url=url, title=title, content=text, datetime=date)

        return article

    except AttributeError:
        return Article()

In [ ]:
# плюс сам парсинг на bs4 довольно ресурсоемкий, поэтому решил его распараллелить.
# Это не I/O bound задача, а скорее вычислительная, поэтому скорость должна вырасти именно засчет применения бОльше кол-ва процессов.

def parallel_parsing_articles(responses_data):
    with Pool() as pool:
        articles = pool.starmap(parse_article, responses_data)
    return articles

def parallel_parsing_articles2(responses_data):
    with mPool() as pool:
        articles = pool.starmap(parse_article, responses_data)
    return articles

In [ ]:
# непосредственно применение написанных функций
articles = {topic: [] for topic in art_dct}

chunk_size = 20
max_articles = 100
for topic in tqdm(list(art_dct)):
    for i in tqdm(range(0, len(art_dct[topic]), chunk_size)):
        results = asyncio.run(fetch_all(art_dct[topic][i:i+chunk_size]))
        articles[topic].extend(parallel_parsing_articles2(results))
        time.sleep(10)

In [ ]:
# перевод из объектов Article в словари
# удаление статей, на которые не удалость попасть
print([(a, len(articles[a])) for a in articles])
articles_dict = {topic: [art.__dict__ for art in articles[topic]] for topic in articles}
articles_dict_clean = {topic: [art for art in articles_dict[topic] if art['url'] is not None] for topic in articles_dict}
print('-'*100)
print([(a, len(articles_dict_clean[a])) for a in articles_dict_clean])

In [ ]:
# склейка датасетов по разным топикам в один с обозначением топика в качестве дополнительного столбца
res_art_lst = []
for topic in articles_dict_clean:
    tmp_df = pd.DataFrame(articles_dict_clean[topic])
    tmp_df['topic'] = topic
    res_art_lst.append(tmp_df)

res_art_df = pd.concat(res_art_lst)
res_art_df['datetime'] = res_art_df['datetime'].dt.strftime("%Y-%m-%d")

In [ ]:
topic_name_to_lenta_id = {
    'Общество/Россия': 1,
    'Силовые структуры': 37,
    'Бывший СССР': 3,
    'Экономика': 4,
    'Наука и техника': 5,
    'Спорт': 8,
    'Туризм/Путешествия': 48,
    'Здоровье': 87,
    'Строительство': 6
}

res_art_df['topic_id'] = res_art_df['topic'].map(topic_name_to_lenta_id)

In [ ]:
res_art_df.to_excel('rbc_arts.xlsx')

## Итог

In [ ]:
print(tbl.info())
tbl.head()

In [ ]:
print(res_art_df.info())
res_art_df.head()